# Generate Synthetic text summarization with Gretel GPT

* In this notebook we use Gretel GPT with Llama-2 7b model to create synthetic text summerization dataset.  
* To run this notebook, you will need an API key from the [Gretel Console](https://console.gretel.ai/users/me/key/).

## Getting Started

In [ ]:
%%capture
!pip install -U gretel-client

In [ ]:
#import required packages
import pandas as pd
from gretel_client import Gretel

## Load and preview training data

In [ ]:
pd.set_option('max_colwidth', None)

# Specify a dataset to train on
DATASET_PATH = 'https://gretel-datasets.s3.us-west-2.amazonaws.com/Text-dataset/Samsum-text-summerization-sample-1000.csv'
df = pd.read_csv(DATASET_PATH)

#Let's look at the training dataset:
df.head()

## Configure and Train the Synthetic Model:

We can experiment different "steps" parameters which result in a change of text SQS.

In [ ]:


PROJECT = 'data-summarization'
LLM = "meta-llama/Llama-2-7b-hf"

gretel = Gretel(project_name=f"{PROJECT}-llama-2-7b", api_key="prompt", validate=True)

trained = gretel.submit_train(
    "natural-language",
    data_source=df,
    pretrained_model=LLM,
    params={"steps": 1000}, 
 )

## Display Text Synthetic Quality Score:

In [ ]:
trained.report.quality_scores

In [ ]:
trained.report.display_in_notebook()